# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# your code here

orders = pd.read_csv('Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [13]:
# your code here Sb1:

customer_view = orders.groupby(['Country','CustomerID']).sum()

customer_view.head()

Unnamed: 0  InvoiceNo     year  month   day  hour  \
Country   CustomerID                                                      
Australia 12386           228213    5381968    20102     98    32    96   
          12388         23878866   55733658   201100    592   381  1230   
          12393         12849375   35452768   128704    315   213   732   
          12415        164201777  398543981  1439876   4254  2169  8061   
          12422          3428310   11563488    42231     85    47   189   

                      Quantity  UnitPrice  amount_spent  
Country   CustomerID                                     
Australia 12386            354      23.91        401.90  
          12388           1462     277.77       2780.66  
          12393            816     145.90       1582.60  
          12415          77670    2097.08     124914.53  
          12422            195      51.12        386.20

In [14]:
customer_view['amount_spent'].max()

280206.01999999984

In [15]:
check_95 = np.percentile(customer_view['amount_spent'], 95)
print(check_95)

5774.277999999997


In [16]:
check_75 = np.percentile(customer_view['amount_spent'], 75)
print(check_75)

1661.1950000000002


In [20]:
# your code here Sb2:
#PLEASE NOTE ORDER MATTERS
customer_view.loc[(customer_view['amount_spent'] >= check_75),'customer_class']='Preferred'
customer_view.loc[(customer_view['amount_spent'] >= check_95),'customer_class']='VIP'
customer_view.loc[(customer_view['amount_spent'] < check_75),'customer_class']= 0

In [18]:
customer_view.head()

Unnamed: 0  InvoiceNo     year  month   day  hour  \
Country   CustomerID                                                      
Australia 12386           228213    5381968    20102     98    32    96   
          12388         23878866   55733658   201100    592   381  1230   
          12393         12849375   35452768   128704    315   213   732   
          12415        164201777  398543981  1439876   4254  2169  8061   
          12422          3428310   11563488    42231     85    47   189   

                      Quantity  UnitPrice  amount_spent customer_class  
Country   CustomerID                                                    
Australia 12386            354      23.91        401.90              0  
          12388           1462     277.77       2780.66      Preferred  
          12393            816     145.90       1582.60              0  
          12415          77670    2097.08     124914.53            VIP  
          12422            195      51.12        386.20              0

In [10]:
# your code here Sb3:
#customer_view['customer_class'] == 'High' &

vip = customer_view.loc[(customer_view['customer_class'] == 'VIP')]
preferred = customer_view.loc[(customer_view['customer_class'] == 'Preferred')]

In [10]:
customer_view.head(100000)


,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent,customer_class
CustomerID,,,,,,,,,,
12346,61619,541431,2011,1,2,10,74215,1.04,77183.60,VIP
12347,42441700,101296926,365971,1383,441,2219,2458,481.21,4310.00,Preferred
12348,2807120,16869685,62324,257,111,472,2341,178.71,1797.24,Preferred
12349,35444274,42165457,146803,803,73,657,631,605.10,1757.55,Preferred
12350,1365627,9231629,34187,34,51,272,197,65.30,334.40,0
...,...,...,...,...,...,...,...,...,...,...
18280,1110495,5457120,20110,30,10,90,45,47.65,180.60,0
18281,1560699,3895248,14077,42,49,70,54,39.36,80.82,0
18282,4642134,6838540,24132,116,60,146,103,62.39,178.05,0


In [21]:
customer_view.groupby(['Country','customer_class']).count()

Unnamed: 0  InvoiceNo  year  month   day  \
Country              customer_class                                             
Australia            0                        5          5     5      5     5   
                     Preferred                3          3     3      3     3   
                     VIP                      1          1     1      1     1   
Austria              0                       10         10    10     10    10   
                     Preferred                1          1     1      1     1   
...                                         ...        ...   ...    ...   ...   
United Arab Emirates 0                        2          2     2      2     2   
United Kingdom       0                     2987       2987  2987   2987  2987   
                     Preferred              756        756   756    756   756   
                     VIP                    178        178   178    178   178   
Unspecified          0                        4          4     4      4     4   

                                     hour  Quantity  UnitPrice  amount_spent  
Country              customer_class                                           
Australia            0                  5         5          5             5  
                     Preferred          3         3          3             3  
                     VIP                1         1          1             1  
Austria              0                 10        10         10            10  
                     Preferred          1         1          1             1  
...                                   ...       ...        ...           ...  
United Arab Emirates 0                  2         2          2             2  
United Kingdom       0               2987      2987       2987          2987  
                     Preferred        756       756        756           756  
                     VIP              178       178        178           178  
Unspecified          0                  4         4          4             4  

[75 rows x 9 columns]

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [32]:
# your code here, shortest by highest to lowest
vip = customer_view.loc[(customer_view['customer_class'] == 'VIP')]
vip_order = vip.groupby(['Country','customer_class']).count().sort_values(by=['InvoiceNo'], ascending=False)

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [33]:
# your code here, using count of values for random column
preferred = customer_view.loc[(customer_view['customer_class'] == 'Preferred')]

prefer_order =preferred.groupby(['Country','customer_class']).count().sort_values(by=['InvoiceNo'], ascending=False)

In [34]:
merged = pd.merge(vip_order,prefer_order, left_on='Country',right_on='Country')

In [35]:
merged.head()

,Unnamed: 0_x,InvoiceNo_x,year_x,month_x,day_x,hour_x,Quantity_x,UnitPrice_x,amount_spent_x,Unnamed: 0_y,InvoiceNo_y,year_y,month_y,day_y,hour_y,Quantity_y,UnitPrice_y,amount_spent_y
Country,,,,,,,,,,,,,,,,,,
United Kingdom,178,178,178,178,178,178,178,178,178,756,756,756,756,756,756,756,756,756
Germany,11,11,11,11,11,11,11,11,11,28,28,28,28,28,28,28,28,28
France,9,9,9,9,9,9,9,9,9,20,20,20,20,20,20,20,20,20
Switzerland,3,3,3,3,3,3,3,3,3,6,6,6,6,6,6,6,6,6
Spain,2,2,2,2,2,2,2,2,2,5,5,5,5,5,5,5,5,5


In [36]:
merged['Total'] = merged['InvoiceNo_x'] + merged['InvoiceNo_y']
merged.head()

,Unnamed: 0_x,InvoiceNo_x,year_x,month_x,day_x,hour_x,Quantity_x,UnitPrice_x,amount_spent_x,Unnamed: 0_y,InvoiceNo_y,year_y,month_y,day_y,hour_y,Quantity_y,UnitPrice_y,amount_spent_y,Total
Country,,,,,,,,,,,,,,,,,,,
United Kingdom,178,178,178,178,178,178,178,178,178,756,756,756,756,756,756,756,756,756,934
Germany,11,11,11,11,11,11,11,11,11,28,28,28,28,28,28,28,28,28,39
France,9,9,9,9,9,9,9,9,9,20,20,20,20,20,20,20,20,20,29
Switzerland,3,3,3,3,3,3,3,3,3,6,6,6,6,6,6,6,6,6,9
Spain,2,2,2,2,2,2,2,2,2,5,5,5,5,5,5,5,5,5,7


In [38]:
merged = merged['Total']

In [39]:
merged.head()

Country
United Kingdom    934
Germany            39
France             29
Switzerland         9
Spain               7
Name: Total, dtype: int64